#### 英文数据库

In [10]:
from datasets import load_dataset

##AI-MO___numina_math-co_t数据集
dataset = load_dataset(r"/root/.cache/huggingface/datasets/AI-MO___numina_math-co_t")
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 859494 examples [00:04, 176230.93 examples/s]
Generating test split: 100 examples [00:00, 26628.81 examples/s]


DatasetDict({
    train: Dataset({
        features: ['source', 'problem', 'solution', 'messages'],
        num_rows: 859494
    })
    test: Dataset({
        features: ['source', 'problem', 'solution', 'messages'],
        num_rows: 100
    })
})


In [14]:
print(dataset['train'][0:1])

{'source': ['synthetic_math'], 'problem': ['Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$.'], 'solution': ['For an arithmetic sequence, the difference between consecutive terms must be equal. Therefore, we can set up the following equations based on the sequence given:\n\\[ (y + 2) - \\left(-\\frac{1}{3}\\right) = 4y - (y+2) \\]\n\nSimplify and solve these equations:\n\\[ y + 2 + \\frac{1}{3} = 4y - y - 2 \\]\n\\[ y + \\frac{7}{3} = 3y - 2 \\]\n\\[ \\frac{7}{3} + 2 = 3y - y \\]\n\\[ \\frac{13}{3} = 2y \\]\n\\[ y = \\frac{13}{6} \\]\n\nThus, the value of $y$ that satisfies the given arithmetic sequence is $\\boxed{\\frac{13}{6}}$.'], 'messages': [[{'content': 'Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$.', 'role': 'user'}, {'content': 'For an arithmetic sequence, the difference between consecutive terms must be equal. Therefore, we can set up the following equations based on the sequence 

## 英文有概率翻译成中文

In [22]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
cache_dir = "/root/autodl-tmp/model/GLM4-9bchat"  # 修改为你自己的缓存目录

############################################
# 1. 加载GLM4-9b-chat模型和tokenizer
############################################
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    trust_remote_code=True,
    cache_dir=cache_dir
)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    cache_dir=cache_dir
).to(device).eval()

print("Model and tokenizer loaded on device:", device)

############################################
# 2. 加载数据集
############################################
ds2 = load_dataset("AI-MO/NuminaMath-CoT")
# ds2包含train和test两个split
print(ds2)
# DatasetDict({
#     train: Dataset({
#         features: ['source', 'problem', 'solution', 'messages'],
#         num_rows: 859494
#     })
#     test: Dataset({
#         features: ['source', 'problem', 'solution', 'messages'],
#         num_rows: 100
#     })
# })

############################################
# 3. 定义翻译函数 translate_text
############################################
def translate_text(text):
    """
    使用ZhipuAI/glm-4-9b-chat将输入文本从英文翻译成中文。
    处理列表类型时，将其join成字符串。
    """
    # 如果text是列表，则拼接成一个字符串
    if isinstance(text, list):
        text = " ".join(str(x) for x in text if x)

    # 如果依旧不是字符串，就做强制转换
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()
    if not text:
        return ""

    # 构造对话消息：系统提示 + 用户消息
    system_message = {
        "role": "system",
        "content": "你是一位翻译助手，请将以下英文文本翻译成中文，保留数学公式与代码不变。"
    }
    user_message = {
        "role": "user",
        "content": text
    }
    messages = [system_message, user_message]

    # 格式化
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=1024,      # 可根据文本长度调大
            do_sample=False,
            top_k=1
        )
        # 移除输入部分
        output_ids = outputs[:, inputs['input_ids'].shape[1]:]
        translated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return translated_text.strip()

############################################
# 4. 批量翻译函数，用于dataset.map()
############################################
def translate_batch(samples):
    """
    samples 是一个batch，每个字段都是列表。
    返回dict时，要包含与输入数量相同的元素列表。
    """
    source_list = samples["source"]
    problem_list = samples["problem"]
    solution_list = samples["solution"]
    messages_list = samples["messages"]

    source_zh = []
    problem_zh = []
    solution_zh = []
    messages_zh = []

    for i in range(len(source_list)):
        source_zh.append(translate_text(source_list[i]))
        problem_zh.append(translate_text(problem_list[i]))
        solution_zh.append(translate_text(solution_list[i]))
        messages_zh.append(translate_text(messages_list[i]))

    return {
        "source_zh": source_zh,
        "problem_zh": problem_zh,
        "solution_zh": solution_zh,
        "messages_zh": messages_zh
    }

############################################
# 5. 对 train/test split 批量翻译
############################################
# 注意：数据量很大，单次map会耗时和资源极高。先在少量数据上测试！
# 例如只翻译前100条：
# ds2_small = ds2["train"].select(range(100))

# 这里直接示范对 train 做map。请根据你实际硬件资源慎重决定！
print("Starting translation of train split... (this might take a LONG time!)")
ds2_train_zh = ds2["train"].map(
    translate_batch,
    batched=True,
    batch_size=2,  # 可调大/调小，看显存和速度
    desc="Translating train split"
)

print("Starting translation of test split...")
ds2_test_zh = ds2["test"].map(
    translate_batch,
    batched=True,
    batch_size=2,
    desc="Translating test split"
)

# 合并
ds2_zh = ds2.copy()
ds2_zh["train"] = ds2_train_zh
ds2_zh["test"] = ds2_test_zh

############################################
# 6. 打印部分翻译结果 & 保存
############################################

# 随机查看train前20条翻译结果
print("\n===== Sample translations from train split =====")
for i in range(20):
    print(f"---- Sample {i} ----")
    print("Original [source]:", ds2['train'][i]['source'])
    print("Translated [source_zh]:", ds2_zh['train'][i]['source_zh'])
    print()
    print("Original [problem]:", ds2['train'][i]['problem'])
    print("Translated [problem_zh]:", ds2_zh['train'][i]['problem_zh'])
    print("================================================\n")

# 保存到本地
save_path = "/root/autodl-tmp/code/2025service_creativity/process_dataset/NuminaMath-CoT-zh"
ds2_zh.save_to_disk(save_path)
print(f"Translated dataset saved to: {save_path}")


Loading tokenizer...
Loading model...


Loading checkpoint shards: 100%|██████████| 10/10 [00:02<00:00,  3.53it/s]


Model and tokenizer loaded on device: cuda
DatasetDict({
    train: Dataset({
        features: ['source', 'problem', 'solution', 'messages'],
        num_rows: 859494
    })
    test: Dataset({
        features: ['source', 'problem', 'solution', 'messages'],
        num_rows: 100
    })
})
Starting translation of train split... (this might take a LONG time!)


Translating train split:   0%|          | 10/859494 [07:12<10326:29:01, 43.25s/ examples]


ValueError: Input length of input_ids is 1241, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
save_path = "/root/autodl-tmp/NuminaMath-CoT-zh"
ds2_zh.save_to_disk(save_path)
print(f"Saved translated dataset to {save_path}")
